In [1]:
import pathlib
import os

import h5py
import pandas as pd
import json
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import dask.array as da
import dask.dataframe as dd
import skimage 
from glob import glob

import cloudpickle
import sklearn
from sklearn.linear_model import LinearRegression as LinReg


import SessionTools.two_photon as st2p


%load_ext autoreload
%autoreload 2 

%matplotlib inline

In [13]:
genotype = "R60D05LexA_jGCaMP7f_EL_CsChrimsontdT"
basedir = pathlib.Path(f'/media/mplitt/SSD_storage/2PData/{genotype}')

for date_dir in basedir.glob('*'):
    date = date_dir.stem
    for fly_dir in date_dir.glob('fly*'): 
        fly = fly_dir.stem
        
        for sess_dir in fly_dir.glob("*"):
            if sess_dir.is_dir() and "SingleImage" not in sess_dir.stem:
                sess = sess_dir.stem
                
                basename_input = pathlib.Path(sess_dir.joinpath(sess))
                metadata = st2p.preprocessing.bruker_metadata.read(basename_input)

                h5name = f'/media/mplitt/SSD_storage/2P_scratch/{genotype}/{date}/{fly}/{sess}/data.h5'
                if not os.path.exists(h5name):
                    tiff_data = st2p.preprocessing.tiff_tools.read(basename_input, 
                                                            metadata['size'],metadata['layout'], first_chan=1)
                    st2p.preprocessing.tiff_tools.convert_to_hdf5(tiff_data,h5name, overwrite=True)
            
                
                f = h5py.File(h5name)
                data = f['/data'][:]

                ref_img = st2p.preprocessing.motion_correction.make_ref_img(data,0)
                data_corr, shifts, error, diffphase = st2p.preprocessing.motion_correction.align_data_chunk(data, 
                                                                                                            ref_img[0,:,:,:], 
                                                                                                            in_place=False)

                napari_outputs_file = f'/media/mplitt/SSD_storage/2P_scratch/{genotype}/{date}/{fly}/{sess}/napari.pkl'

                with open(napari_outputs_file, 'rb') as file:
                    np_layers = cloudpickle.load(file)
                masks = np_layers['rois']
                bckgnd = np_layers['background']

                csv_files = glob(f'/media/mplitt/SSD_storage/2PData/{genotype}/{date}/{fly}/{sess}/*.csv')
                vr_file = pathlib.Path(csv_files[0])
                df = dd.read_csv(vr_file).compute()

                frame_times = np.array(metadata['frame_times']).mean(axis=-1)*1000
                df_aligned = st2p.preprocessing.signals.align_vr_2p(df,frame_times)

                F, notF = st2p.preprocessing.signals.extract_2p_timeseries(data_corr, masks, 16, bckgnd_mask = bckgnd, max_proj=False) 
                F_bleed_corr = 0*F
                for ch in range(F.shape[0]):
                    lr = LinReg().fit(notF[ch,np.newaxis, :].T, F[ch,:,:].T)
                    F_bleed_corr[ch,:,:] = F[ch,:,:]-lr.predict(notF[ch,np.newaxis,:].T).T  

                F_sm = sp.ndimage.gaussian_filter1d(F_bleed_corr,1.5,axis=-1)
                F_sm = sp.ndimage.gaussian_filter1d(F_sm,.5,axis=1, mode='wrap')

                dff = sp.stats.zscore(F_sm[1,:,:],axis=-1)

                heading = np.angle(np.exp(1j*(np.pi-df_aligned[' Heading'].to_numpy().ravel())))

                bin_centers =  np.linspace(-np.pi, np.pi, num=16)[:,np.newaxis]
                dff_complex = dff*np.cos(bin_centers) + 1j*dff*np.sin(bin_centers)
                rho, phi = np.abs(dff_complex.mean(axis=0)), np.angle(dff_complex.mean(axis=0))
                
                data = {'fly': fly,
                        'date': date,
                        'session': sess,
                        'F': F,
                        'not_F': notF,
                        'F_bleed_corr':F_bleed_corr,
                        'F_sm': F_sm,
                        'df_aligned': df_aligned,
                        'dff': dff,
                        'heading': heading,
                        'bin_centers': bin_centers,
                        'dff_complex': dff_complex,
                        'rho': rho,
                        'phi': phi}
                sess_pkl_dir = f'/media/mplitt/SSD_storage/2P_scratch/{genotype}/{date}/{fly}/{sess}/sess.pkl'
                with open(sess_pkl_dir, 'wb') as file:
                    cloudpickle.dump(data, file)
                

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [8]:
a = [2 + 2j , 1 + 1j]
print(np.mean(a))

(1.5+1.5j)


In [13]:
dates = glob(basedir + '/*')
print(dates)

['/media/mplitt/SSD_storage/2PData/R60D05LexA_jGCaMP7f_EL_CsChrimsontdT/02_11_2023', '/media/mplitt/SSD_storage/2PData/R60D05LexA_jGCaMP7f_EL_CsChrimsontdT/17_10_2023', '/media/mplitt/SSD_storage/2PData/R60D05LexA_jGCaMP7f_EL_CsChrimsontdT/31_10_2023']
